# Table 2 RHS - Training models on IN9L

In this section, we train deep learning models on the IN9L data set provided by the paper. This is essentially the ImageNet dataset but reduced to only 9 classes. Given its similarity to ImageNet, we will use pretrained models given by PyTorch, and train only the final fully connected layer such that the output is logits of 9 classes.

Results:

Test set | ResNet-50 | AlexNet | ShuffleNet
--- | --- | --- | ---
original | 98.2% | 93.8% | 88.4%
only_bg_t | 47.0% | 42.1% | 35.0%
mixed_same | 94.4% | 83.9% | 81.3%
mixed_rand | 82.8% | 63.6% | 63.1%
BG-Gap | 11.6% | 20.3% | 18.2%

In [1]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer
    import torchbearer

     |████████████████████████████████| 143kB 11.2MB/s 


In [2]:
# automatically reload external modules if they change
%load_ext autoreload
%autoreload 2

import torch.nn.functional as F
import torchvision.transforms as transforms
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchbearer import Trial
from torchvision.datasets import ImageFolder
import os
import json
import torchvision
import numpy as np

## Define the dataset
Here we extract the tar.gz file containing the IN9L training data. This takes about 6 minutes to extract.
We also store the data as a pytorch DataLoader, first transforming the images such that they are all of identical dimensions. 

In [3]:
!tar -xvf drive/MyDrive/ReproducabilityChallenge/in9l.tar.gz

Streaming output truncated to the last 5000 lines.
in9l/train/00_dog/n02112137_15846.JPEG
in9l/train/00_dog/n02096051_1284.JPEG
in9l/train/00_dog/n02107908_3980.JPEG
in9l/train/00_dog/n02091635_7767.JPEG
in9l/train/00_dog/n02109961_16117.JPEG
in9l/train/00_dog/n02089867_7001.JPEG
in9l/train/00_dog/n02105251_5637.JPEG
in9l/train/00_dog/n02105855_7397.JPEG
in9l/train/00_dog/n02107683_3080.JPEG
in9l/train/00_dog/n02105412_8023.JPEG
in9l/train/00_dog/n02112137_1402.JPEG
in9l/train/00_dog/n02090379_5031.JPEG
in9l/train/00_dog/n02110806_10220.JPEG
in9l/train/00_dog/n02104029_3758.JPEG
in9l/train/00_dog/n02097474_12183.JPEG
in9l/train/00_dog/n02107574_2131.JPEG
in9l/train/00_dog/n02100236_10897.JPEG
in9l/train/00_dog/n02091244_746.JPEG
in9l/train/00_dog/n02096177_12822.JPEG
in9l/train/00_dog/n02104029_918.JPEG
in9l/train/00_dog/n02100735_2168.JPEG
in9l/train/00_dog/n02113799_1439.JPEG
in9l/train/00_dog/n02113186_9957.JPEG
in9l/train/00_dog/n02093428_9383.JPEG
in9l/train/00_dog/n02085620_7435.

In [4]:
transform = transforms.Compose([
            transforms.Resize(224),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),
        ])

batch_size = 128

train_dataset = ImageFolder("in9l/train", transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = ImageFolder("in9l/val", transform)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

## ResNet-50
We take a pre-trained (on ImageNet) ResNet-50 and change the final fully connected layer such that the output is 9 classes instead of 1000. We then only train this final layer since the IN9L is very similar to ImageNet. 

In [ ]:
#Change fully connected layer to output the desired 9 classes, instead of 1000
from torchvision.models import resnet50
model = resnet50(pretrained=True)
#model.avgpool = nn.AdaptiveAvgPool2d((1,1))
model.fc = nn.Linear(2048, len(train_dataset.classes))
model.train()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
# Freeze layers by not tracking gradients
for param in model.parameters():
    param.requires_grad = False
model.fc.weight.requires_grad = True #unfreeze last layer weights
model.fc.bias.requires_grad = True #unfreeze last layer biases

optimiser = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4) #only optimse non-frozen layers

In [ ]:
# define the loss function and the optimiser
loss_function = nn.CrossEntropyLoss()

#device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy']).to('cuda:0')
trial.with_generators(train_loader, val_generator=val_loader)
trial.run(epochs=3)
results = trial.evaluate(data_key=torchbearer.VALIDATION_DATA)
print()
print(results)



{'val_loss': 0.09437453746795654, 'val_acc': 0.9706944823265076}


In [ ]:
#save the trained model weights
torch.save(model.to('cpu').state_dict(), "./resnetin9l.weights")

## AlexNet

In [14]:
#Change fully connected layer to output the desired 9 classes, instead of 1000
alexnet = torch.hub.load('pytorch/vision:v0.9.0', 'alexnet', pretrained=True)
alexnet.classifier[6] = nn.Linear(4096, len(train_dataset.classes))
alexnet.train()

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [16]:
# Freeze layers by not tracking gradients
for param in alexnet.parameters():
    param.requires_grad = False
alexnet.classifier[6].weight.requires_grad = True #unfreeze last layer weights
alexnet.classifier[6].bias.requires_grad = True #unfreeze last layer biases

optimiser = optim.Adam(filter(lambda p: p.requires_grad, alexnet.parameters()), lr=1e-4) #only optimse non-frozen layers

In [17]:
# define the loss function and the optimiser
loss_function = nn.CrossEntropyLoss()

#device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(alexnet, optimiser, loss_function, metrics=['loss', 'accuracy']).to('cuda:0')
trial.with_generators(train_loader, val_generator=val_loader)
trial.run(epochs=3)
results = trial.evaluate(data_key=torchbearer.VALIDATION_DATA)
print()
print(results)



{'val_loss': 0.2854683995246887, 'val_acc': 0.904722273349762}


In [18]:
#save the trained model weights
torch.save(alexnet.to('cpu').state_dict(), "./alexnetin9l.weights")

## ShuffleNet

In [39]:
import torchvision.models as models
shufflenet = models.shufflenet_v2_x1_0(pretrained=True)
shufflenet.fc = nn.Linear(1024, len(train_dataset.classes))
shufflenet.train()

Downloading: "https://download.pytorch.org/models/shufflenetv2_x1-5666bf0f80.pth" to /root/.cache/torch/hub/checkpoints/shufflenetv2_x1-5666bf0f80.pth


ShuffleNetV2(
  (conv1): Sequential(
    (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (stage2): Sequential(
    (0): InvertedResidual(
      (branch1): Sequential(
        (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=24, bias=False)
        (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Conv2d(24, 58, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (3): BatchNorm2d(58, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): ReLU(inplace=True)
      )
      (branch2): Sequential(
        (0): Conv2d(24, 58, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(58, eps=1e-05, momentum=0.1, affine=True, track_running_

In [40]:
# Freeze layers by not tracking gradients
for param in shufflenet.parameters():
    param.requires_grad = False
shufflenet.fc.weight.requires_grad = True #unfreeze last layer weights
shufflenet.fc.bias.requires_grad = True #unfreeze last layer biases

optimiser = optim.Adam(filter(lambda p: p.requires_grad, shufflenet.parameters()), lr=1e-4) #only optimse non-frozen layers

In [41]:
# define the loss function and the optimiser
loss_function = nn.CrossEntropyLoss()

#device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(shufflenet, optimiser, loss_function, metrics=['loss', 'accuracy']).to('cuda:0')
trial.with_generators(train_loader, val_generator=val_loader)
trial.run(epochs=3)
results = trial.evaluate(data_key=torchbearer.VALIDATION_DATA)
print()
print(results)



{'val_loss': 1.2517708539962769, 'val_acc': 0.8544444441795349}


In [42]:
#save the trained model weights
torch.save(shufflenet.to('cpu').state_dict(), "./shufflenetin9l.weights")

## Evaluate Model

In [19]:
def getDataLoader(datadir = 'mixed_same/val'):
    # convert each image to tensor format
    preprocess = transforms.Compose([transforms.Resize(224),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(), 
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

    #datadir = '/Volumes/DRIVE/test_sets/mixed_same/val'
    #datadir = os.path.join('/Volumes/DRIVE/test_sets/mixed_same/', 'val')
    imgFolder = ImageFolder(datadir, transform=preprocess)
    testloader = DataLoader(imgFolder, batch_size=32, shuffle=False)
    return testloader

In [20]:
def IN9LAccuracy(model, data):
  correct = 0
  total = 0

  for input, label in data:
    input, label = input.to('cuda:0'), label.to('cuda:0')
    output = model(input).detach()
    output = torch.argmax(output, dim=1)
    correct += torch.sum(output==label).item()
    total += len(output)

  print("Test Accuracy:", correct/total*100, "%")


In [21]:
!unzip drive/MyDrive/ReproducabilityChallenge/only_bg_t.zip
!unzip drive/MyDrive/ReproducabilityChallenge/mixed_rand.zip
!unzip drive/MyDrive/ReproducabilityChallenge/original.zip
!unzip drive/MyDrive/ReproducabilityChallenge/mixed_same.zip

Streaming output truncated to the last 5000 lines.
  inflating: mixed_same/val/04_carnivore/fg_n02115641_41341_bg_n02233338_15470.JPEG  
  inflating: __MACOSX/mixed_same/val/04_carnivore/._fg_n02115641_41341_bg_n02233338_15470.JPEG  
  inflating: mixed_same/val/04_carnivore/fg_n02128757_04135_bg_n02279972_20458.JPEG  
  inflating: __MACOSX/mixed_same/val/04_carnivore/._fg_n02128757_04135_bg_n02279972_20458.JPEG  
  inflating: mixed_same/val/04_carnivore/fg_n02128757_22170_bg_n02264363_47939.JPEG  
  inflating: __MACOSX/mixed_same/val/04_carnivore/._fg_n02128757_22170_bg_n02264363_47939.JPEG  
  inflating: mixed_same/val/04_carnivore/fg_n02119022_00157_bg_n02177972_01737.JPEG  
  inflating: __MACOSX/mixed_same/val/04_carnivore/._fg_n02119022_00157_bg_n02177972_01737.JPEG  
  inflating: mixed_same/val/04_carnivore/fg_n02119022_06668_bg_n02276258_45550.JPEG  
  inflating: __MACOSX/mixed_same/val/04_carnivore/._fg_n02119022_06668_bg_n02276258_45550.JPEG  
  inflating: mixed_same/val/04_car

In [22]:
#Get the original dataset
original = getDataLoader(datadir = 'original/val')

In [23]:
#Get the only_bg_t dataset
only_bg_t = getDataLoader(datadir = 'only_bg_t/val')

In [24]:
#Get the mixed_same dataset
mixed_same = getDataLoader(datadir = 'mixed_same/val')

In [25]:
#Get the mixed_rand dataset
mixed_rand = getDataLoader(datadir = 'mixed_rand/val')

## ResNet-50 Evaluation

In [ ]:
model.to('cuda:0')
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
IN9LAccuracy(model, original)

Test Accuracy: 98.17283950617283 %


In [ ]:
IN9LAccuracy(model, only_bg_t)

Test Accuracy: 47.01234567901235 %


In [ ]:
IN9LAccuracy(model, mixed_same)

Test Accuracy: 94.39506172839506 %


In [ ]:
IN9LAccuracy(model, mixed_rand)

Test Accuracy: 82.79012345679013 %


## AlexNet Evaluation

In [29]:
alexnet.to('cuda:0')
alexnet.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [30]:
IN9LAccuracy(alexnet, original)

Test Accuracy: 93.77777777777779 %


In [31]:
IN9LAccuracy(alexnet, only_bg_t)

Test Accuracy: 42.098765432098766 %


In [32]:
IN9LAccuracy(alexnet, mixed_same)

Test Accuracy: 83.87654320987654 %


In [33]:
IN9LAccuracy(alexnet, mixed_rand)

Test Accuracy: 63.60493827160494 %


## ShuffleNet Evaluation

In [43]:
shufflenet.to('cuda:0')
shufflenet.eval()

ShuffleNetV2(
  (conv1): Sequential(
    (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (stage2): Sequential(
    (0): InvertedResidual(
      (branch1): Sequential(
        (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=24, bias=False)
        (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Conv2d(24, 58, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (3): BatchNorm2d(58, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): ReLU(inplace=True)
      )
      (branch2): Sequential(
        (0): Conv2d(24, 58, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(58, eps=1e-05, momentum=0.1, affine=True, track_running_

In [44]:
IN9LAccuracy(shufflenet, original)

Test Accuracy: 88.41975308641975 %


In [45]:
IN9LAccuracy(shufflenet, only_bg_t)

Test Accuracy: 34.96296296296296 %


In [46]:
IN9LAccuracy(shufflenet, mixed_same)

Test Accuracy: 81.33333333333333 %


In [47]:
IN9LAccuracy(shufflenet, mixed_rand)

Test Accuracy: 63.11111111111111 %
